In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import re
import seaborn as sns
from tqdm import tqdm
import nltk
import random
nltk.download('punkt')
from nltk.tokenize import word_tokenize,sent_tokenize
import pickle

train_example_names = [fn.split('.')[0] for fn in os.listdir('data/train')]
test_example_names = [fn.split('.')[0] for fn in os.listdir('data/test')]

metadata = pd.read_csv('data/train.csv')
docIdx = train_example_names.copy()

names_df = pd.read_csv('data/babynames-clean.csv', header = None, names = ['name', 'gender'])
names_list = list(set(names_df.name.values))
names_list = [n.lower() for n in names_list]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ozano\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def load_train_example_by_name(name):
    doc_path = os.path.join('data/train', name + '.json')
    with open(doc_path) as f:
        data = json.load(f)
    return data

def text_cleaning(text):
    text = re.sub('[^A-Za-z0-9]+', ' ', str(text)).strip() # remove unnecessary literals

    # remove extra spaces
    text = re.sub("\s+"," ", text)

    return text.strip()

def process_doc(doc_id):
    doc_json = load_train_example_by_name(doc_id)
    doc_text = ' '.join([sec['text'] for sec in doc_json])

    # Tokenize sentencewise
    sentences = sent_tokenize(doc_text)
    return sentences

def get_doc(doc_id):
    sents = process_doc(doc_id)
    return [text_cleaning(s) for s in sents]

In [3]:
connection_tokens = {'s', 'of', 'and', 'in', 'on', 'for', 'from', 'the'}

def group_ok(group_df):
    if len(group_df) < 2:
        return False
    
    nonconn_count = len(group_df) - group_df.is_conn_token.sum()
    if nonconn_count < 2:
        return False

    if group_df.is_long_token.sum() < 2:
        return False

    return True

def get_connected_uppercase(tokens):
    if len(tokens) < 5:
        return []

    sent_df = pd.DataFrame({'token': tokens})
    sent_df['is_conn_token'] = sent_df.token.apply(lambda x: x.lower() in connection_tokens)
    sent_df['is_long_token'] = sent_df.token.str.len() > 4
    sent_df['in_name'] = sent_df.apply(lambda x: x.token[0].isupper() if not x.is_conn_token else True, axis = 1)

    in_name_pieces = []
    groups = sent_df.groupby((sent_df.in_name.shift() != sent_df.in_name).cumsum())
    for name, group in groups:
        if group.in_name.iloc[0]:
            in_name_pieces.append(group)

    return [list(p['token'].values) for p in in_name_pieces if group_ok(p)]

In [4]:
s = get_doc(docIdx[3])[6]
s

'In the'

In [5]:
possible_labels = []

for doc_id in tqdm(docIdx):
    sents = get_doc(doc_id)

    for s in sents:
        pos_labels_sent = get_connected_uppercase(s.split(' '))
        pos_labels_sent = [' '.join(tokens) for tokens in pos_labels_sent]
        possible_labels.extend(pos_labels_sent)

  9%|▊         | 1238/14316 [23:40<4:10:05,  1.15s/it]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "C:\Users\ozano\.conda\envs\torch\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-d6de75641fbf>", line 7, in <module>
    pos_labels_sent = get_connected_uppercase(s.split(' '))
  File "<ipython-input-3-db245db6dd1d>", line 27, in get_connected_uppercase
    for name, group in groups:
  File "C:\Users\ozano\.conda\envs\torch\lib\site-packages\pandas\core\groupby\ops.py", line 148, in get_iterator
    for key, (i, group) in zip(keys, splitter):
  File "C:\Users\ozano\.conda\envs\torch\lib\site-packages\pandas\core\groupby\ops.py", line 970, in __iter__
    sdata = self._get_sort

TypeError: object of type 'NoneType' has no len()

In [31]:
banned_tokens = ['ICT', 'DOT']

possible_labels = list(set(possible_labels))

with open("data/possible_labesl.txt", "w") as f:
    for l in possible_labels:
        for t in banned_tokens:
            if t in l:
                continue

        for n in names_list:
            if n in l.lower():
                continue

        f.write(l + '\n')